In [1]:
import gdown
import zipfile

# Descargar el dataset desde Google Drive
url = "https://drive.google.com/uc?id=1Pqs5Y6dZr4R66Dby5hIUIjPZtBI28rmJ"
gdown.download(url, "natural_scenes.zip", quiet=False)

# Descomprimir el dataset
with zipfile.ZipFile("natural_scenes.zip", 'r') as zip_ref:
    zip_ref.extractall("natural_scenes")


Downloading...
From (original): https://drive.google.com/uc?id=1Pqs5Y6dZr4R66Dby5hIUIjPZtBI28rmJ
From (redirected): https://drive.google.com/uc?id=1Pqs5Y6dZr4R66Dby5hIUIjPZtBI28rmJ&confirm=t&uuid=2e84f744-a82f-45c6-9b8b-0217368849e7
To: /Users/pepeargentoo/AAII-2/natural_scenes.zip
100%|██████████| 363M/363M [01:30<00:00, 4.00MB/s] 


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Definición de generadores de datos con aumento y normalización
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flujo de datos de entrenamiento y validación desde la carpeta correcta
train_generator = train_datagen.flow_from_directory(
    "natural_scenes/seg_train/seg_train", target_size=(150, 150), batch_size=32,
    class_mode='categorical', subset='training')

val_generator = train_datagen.flow_from_directory(
    "natural_scenes/seg_train/seg_train", target_size=(150, 150), batch_size=32,
    class_mode='categorical', subset='validation')

# Flujo de datos de prueba
test_generator = test_datagen.flow_from_directory(
    "natural_scenes/seg_test/seg_test", target_size=(150, 150), batch_size=32,
    class_mode='categorical')


Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(6, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.fit(train_generator, validation_data=val_generator, epochs=10)


Epoch 1/10


/Users/pepeargentoo/AAII-2/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/pepeargentoo/AAII-2/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


351/351 ━━━━━━━━━━━━━━━━━━━━ 107s 302ms/step - accuracy: 0.5139 - loss: 1.5409 - val_accuracy: 0.7536 - val_loss: 0.6911
Epoch 2/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 111s 315ms/step - accuracy: 0.7670 - loss: 0.6506 - val_accuracy: 0.7539 - val_loss: 0.6814
Epoch 3/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 110s 312ms/step - accuracy: 0.8467 - loss: 0.4342 - val_accuracy: 0.7721 - val_loss: 0.6491
Epoch 4/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 108s 306ms/step - accuracy: 0.9273 - loss: 0.2309 - val_accuracy: 0.7710 - val_loss: 0.7883
Epoch 5/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 108s 309ms/step - accuracy: 0.9652 - loss: 0.1141 - val_accuracy: 0.7907 - val_loss: 0.8249
Epoch 6/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 110s 313ms/step - accuracy: 0.9887 - loss: 0.0508 - val_accuracy: 0.7889 - val_loss: 0.8621
Epoch 7/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 110s 312ms/step - accuracy: 0.9935 - loss: 0.0310 - val_accuracy: 0.7832 - val_loss: 0.9858
Epoch 8/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 107s 306ms/step - accuracy: 0.9893 - loss: 0.04

In [ ]:
from tensorflow.keras.layers import Add, Input
from tensorflow.keras.models import Model

def residual_block(x):
    skip = x
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    return Add()([x, skip])

inputs = Input(shape=(150, 150, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
x = residual_block(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(6, activation='softmax')(x)

model_residual = Model(inputs, outputs)
model_residual.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_residual.fit(train_generator, validation_data=val_generator, epochs=10)


Epoch 1/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 712s 2s/step - accuracy: 0.5219 - loss: 3.7659 - val_accuracy: 0.7325 - val_loss: 0.7301
Epoch 2/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 710s 2s/step - accuracy: 0.8160 - loss: 0.5312 - val_accuracy: 0.7386 - val_loss: 0.7330
Epoch 3/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 713s 2s/step - accuracy: 0.9294 - loss: 0.2367 - val_accuracy: 0.7507 - val_loss: 0.8234
Epoch 4/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 716s 2s/step - accuracy: 0.9705 - loss: 0.1054 - val_accuracy: 0.7404 - val_loss: 0.8923
Epoch 5/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 708s 2s/step - accuracy: 0.9868 - loss: 0.0650 - val_accuracy: 0.7429 - val_loss: 1.1448
Epoch 6/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 710s 2s/step - accuracy: 0.9927 - loss: 0.0439 - val_accuracy: 0.7507 - val_loss: 1.0489
Epoch 7/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 713s 2s/step - accuracy: 0.9955 - loss: 0.0296 - val_accuracy: 0.7268 - val_loss: 1.3168
Epoch 8/10
351/351 ━━━━━━━━━━━━━━━━━━━━ 715s 2s/step - accuracy: 0.9924 - loss: 0.0387 - val_accu

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Congelar las capas base

model_transfer = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(6, activation='softmax')
])

model_transfer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_transfer.fit(train_generator, validation_data=val_generator, epochs=10)


In [ ]:
# Evaluar cada modelo en el conjunto de prueba
print("Evaluación Modelo Denso:")
model_dense.evaluate(test_generator)

print("Evaluación Modelo CNN:")
model_cnn.evaluate(test_generator)

print("Evaluación Modelo Residual:")
model_residual.evaluate(test_generator)

print("Evaluación Transfer Learning:")
model_transfer.evaluate(test_generator)
